# Findings and Methodology

## Methodology

Data was taken from the OKX API, using the ccxt package, similar to what was provided. OKX provides historical data via the ohlcv function, or `fetch_ohlcv` when using ccxt. We can also query current prices using the `fetch_ticker` function in ccxt.

For data, the main issue is getting a more rich historical database, to help with backtesting. For the purpose of time, I used a 5minute period interval, though it can be easily upsampled or downsampled. I used a sqlite3 database, to simplify storing data continuously while being able to query the same database. This however, introduces slightly more complexity, since a database would need to be initialized.

The specified problem is a carry trade strategy, where we short/long a futures/spot pair, depending on the prices. The problem also specified using a threshold for trade signals, i.e., a threshold for the futures premium.

For cryptos, we have perpetual and typical futures contracts. Perpetual futures utilizes a 8 hour settlement period for the funding rate for OKX - this can be pulled via the API as well. But to simplify things, we will use a hard coded value for now. Also, a typical carry trade should take into consideration margins, borrow, and lending costs for the full profitability of a trade. Again, we assume these don't exist for now, to keep the strategy as simple as we can will maintaining extendibility.

This library was written in functional programming, and should be refactored naturally when the library grows, likely with a Strategy and Backtest object, along with a more robust OKXTrader object that connects to the OKX trading API for live trades.


# Usage


Steps to use the repo:
1. Clone the repo
2. Create a virtual env, using python 312 - this has only been tested in py312
3. Run `pip install .` using the virtual env
4. Open demo.ipynb to see example usages

## src/spot_futures_arbitrage/db/db_init.py

Usage details:

create_price_table(db): Pass the path to your SQLite database file (e.g., DB_PATH). This will create the prices table if it does not exist.

create_historical_price_table(db): Pass the path to your SQLite database file (e.g., DB_PATH). This will create the historical_prices table if it does not exist.

You typically call these functions at the start of your application or in a setup script to ensure the required tables exist before inserting or querying data.

In [9]:
from spot_futures_arbitrage.db.db_init import create_price_table, create_historical_price_table
from spot_futures_arbitrage.db.db_config import DB_PATH 

# Create the 'prices' table in the database
create_price_table(DB_PATH)

# Create the 'historical_prices' table in the database
create_historical_price_table(DB_PATH)

2025-07-31 00:12:44,071 | INFO | Database and table was created.
2025-07-31 00:12:44,073 | INFO | Database and table was created.



## src/spot_futures_arbitrage/db/data_scraper.py

Usage details:

find_symbol(exchange, base, quote): Returns a list of spot symbols for the given base and quote.

find_future_symbols(exchange, base, quote): Returns a list of futures symbols for the given base and quote.

fetch_and_store(exchange, symbols, futures=0): Fetches ticker data for the given symbols and stores it in the database. Set futures=1 for futures contracts.

fetch_and_store_historical(exchange, symbols, futures=0): Fetches OHLCV historical data for the given symbols and stores it in the database. Set futures=1 for futures contracts.

You typically call these functions in a data ingestion script or scheduler to keep your database updated with the latest and historical price data.

In [2]:
from spot_futures_arbitrage.db.data_scraper import (
    find_symbol,
    find_future_symbols,
    fetch_and_store,
    fetch_and_store_historical,
    spot,
    fut
)

# Find spot and futures symbols for BTC/USDT
spot_symbols = find_symbol(spot, 'BTC', 'USDT')
future_symbols = find_future_symbols(fut, 'BTC', 'USDT')

# Fetch and store latest spot prices for BTC/USDT
fetch_and_store(spot, spot_symbols)

# Fetch and store latest futures prices for BTC/USDT
fetch_and_store(fut, future_symbols, futures=1)

# Fetch and store historical spot prices for BTC/USDT
fetch_and_store_historical(spot, spot_symbols)

# Fetch and store historical futures prices for BTC/USDT
fetch_and_store_historical(fut, future_symbols, futures=1)

2025-07-30 23:24:48,009 | INFO | [2025-07-30T15:24:47.010Z] BTC/USDT: last=118270.6, bid=118271.9, ask=118272.0
2025-07-30 23:24:48,009 | INFO | [2025-07-30T15:24:47.010Z] BTC/USDT: last=118270.6, bid=118271.9, ask=118272.0
2025-07-30 23:24:48,075 | INFO | [2025-07-30T15:24:46.115Z] BTC/USDT:USDT-250801: last=118523.0, bid=118353.7, ask=118374.4
2025-07-30 23:24:48,075 | INFO | [2025-07-30T15:24:46.115Z] BTC/USDT:USDT-250801: last=118523.0, bid=118353.7, ask=118374.4
2025-07-30 23:24:48,177 | INFO | [2025-07-30T15:24:47.409Z] BTC/USDT:USDT-250808: last=118432.2, bid=118471.9, ask=118499.1
2025-07-30 23:24:48,177 | INFO | [2025-07-30T15:24:47.409Z] BTC/USDT:USDT-250808: last=118432.2, bid=118471.9, ask=118499.1
2025-07-30 23:24:48,281 | INFO | [2025-07-30T15:24:47.215Z] BTC/USDT:USDT-250829: last=118918.3, bid=118964.8, ask=118991.2
2025-07-30 23:24:48,281 | INFO | [2025-07-30T15:24:47.215Z] BTC/USDT:USDT-250829: last=118918.3, bid=118964.8, ask=118991.2
2025-07-30 23:24:48,387 | INFO |

['BTC/USDT']
['BTC/USDT:USDT-250801', 'BTC/USDT:USDT-250808', 'BTC/USDT:USDT-250829', 'BTC/USDT:USDT-250926', 'BTC/USDT:USDT-251226', 'BTC/USDT:USDT-260327', 'BTC/USDT:USDT']


2025-07-30 23:24:49,084 | INFO | [2025-07-30T21:45:00] BTC/USDT:USDT-250801: open=117755.5, high=117826.3, low=117755.5, close=117826.3, volume=0.0016
2025-07-30 23:24:49,084 | INFO | [2025-07-30T21:45:00] BTC/USDT:USDT-250801: open=117755.5, high=117826.3, low=117755.5, close=117826.3, volume=0.0016
2025-07-30 23:24:49,085 | INFO | [2025-07-30T21:46:00] BTC/USDT:USDT-250801: open=117826.3, high=117826.3, low=117826.3, close=117826.3, volume=0.0
2025-07-30 23:24:49,085 | INFO | [2025-07-30T21:46:00] BTC/USDT:USDT-250801: open=117826.3, high=117826.3, low=117826.3, close=117826.3, volume=0.0
2025-07-30 23:24:49,086 | INFO | [2025-07-30T21:47:00] BTC/USDT:USDT-250801: open=117826.3, high=117826.3, low=117826.3, close=117826.3, volume=0.0
2025-07-30 23:24:49,086 | INFO | [2025-07-30T21:47:00] BTC/USDT:USDT-250801: open=117826.3, high=117826.3, low=117826.3, close=117826.3, volume=0.0
2025-07-30 23:24:49,087 | INFO | [2025-07-30T21:48:00] BTC/USDT:USDT-250801: open=117826.3, high=117826.3,

## src/spot_futures_arbitrage/strategy.py

Usage details:

carry(...): Calculates the annualized carry for futures or perpetual contracts.

evaluate_trade(...): Evaluates the carry value for a trade based on symbol and prices.

get_prices(): Fetches the latest prices from the database.

get_historical_prices(): Fetches historical prices from the database.

resample_prices(df, freq): Resamples a DataFrame of prices to a specified frequency.

symbol_filter(df, base, quote): Filters a DataFrame for specific base and quote symbols.

calculate_carry(base, quote, ...): Calculates carry signals using the latest data from the DB.

generate_signals(f, base, quote, ...): Generates trading signals based on a strategy function.

carry_strategy(base, quote, ...): Runs the carry strategy and returns a DataFrame with signals.


In [4]:
from spot_futures_arbitrage.strategy import (
    carry,
    evaluate_trade,
    get_prices,
    get_historical_prices,
    resample_prices,
    symbol_filter,
    calculate_carry,
    generate_signals,
    carry_strategy
)

# Calculate carry for a futures contract
annualized_carry = carry(fut_price=41000, spot_price=40000, days_to_expiry=30)

# Calculate carry for a perpetual contract
annualized_carry_perp = carry(fut_price=41000, spot_price=40000, funding_rate=0.0005, perpetual=True)

# Evaluate a trade
carry_value = evaluate_trade('BTC/USDT:USDT', spot_price=40000, fut_price=41000)

# Fetch latest prices from the database
prices_df = get_prices()

# Fetch historical prices from the database
historical_df = get_historical_prices()

# Resample prices DataFrame to 5-minute intervals
resampled_df = resample_prices(historical_df, freq='5min')

# Filter DataFrame for BTC/USDT symbols
filtered_df = symbol_filter(prices_df, base='BTC', quote='USDT')

# Calculate carry signals for BTC/USDT
carry_signals_df = calculate_carry(base='BTC', quote='USDT', timeframe='5min')

# Generate trading signals using a strategy function
signals_df = generate_signals(calculate_carry, base='BTC', quote='USDT', threshold=0.05)

# Run the carry strategy for BTC/USDT
carry_strategy_df = carry_strategy(base='BTC', quote='USDT', timeframe='5min', threshold=0.05)

/Users/benjaminlee/personal/spot_futures_arbitrage/.venv/lib/python3.12/site-packages/spot_futures_arbitrage/strategy.py:100: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .last()  # or use .ohlc(), .mean(), etc.
/Users/benjaminlee/personal/spot_futures_arbitrage/.venv/lib/python3.12/site-packages/spot_futures_arbitrage/strategy.py:100: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .last()  # or use .ohlc(), .m

## src/spot_futures_arbitrage/backtest.py

Usage details:

simple_backtest(df, price_col='close', signal_col='signal'): Runs a simple backtest on a DataFrame with price and signal columns.

run_backtest(df, backtest_func, groupby=[], price_col='close', signal_col='signal', pair_price_col=''): Runs a backtest function on a DataFrame, optionally grouped by specified columns.

summarize_portfolio(df, groupby, pnl_col='pnl', cumulative_pnl_col='cumulative_pnl', date_col='datetime'): Summarizes portfolio performance metrics from a backtest results DataFrame.


In [ ]:
from spot_futures_arbitrage.backtest import (
    simple_backtest,
    run_backtest,
    summarize_portfolio
)
import pandas as pd

# Example DataFrame for backtesting
df = pd.DataFrame({
    'datetime': pd.date_range('2024-01-01', periods=5, freq='D'),
    'close': [100, 102, 101, 103, 104],
    'signal': [0, 1, 1, -1, 0],
    'symbol': ['BTC/USDT'] * 5
})

# Run a simple backtest
backtest_results = simple_backtest(df)

# Run backtest with grouping (if you have multiple symbols)
results = run_backtest(df, simple_backtest, groupby=['symbol'])

# Summarize portfolio performance
summary = summarize_portfolio(backtest_results, groupby=['symbol'])